# Data Preperation
The resulting data from this notebook will combine the positional data (latitude, longitude, town) of the ChemDataForJeffOlson.csv and all of the tables from the Land use survey.
## Merge all tables from the land use survey
Here we run through the files from the land use survey. All files have the same colum names and thus we just need to append them all. I created a dataframe from the first file, then ran through the rest of them turning each into a dataframe then appending it to the first.

In [97]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from urllib.request import urlretrieve
import geopandas as gpd

pd.set_option('display.max_columns', None)

In [98]:
# set the path to the folder with all the tables
land_use_folder = 'assets/Tables'

# set the path to the chem data file
chem_data_file_path = 'assets/ChemDataforJeffOlson.csv'

# set the path to the population data (Optional)
USE_POPULATION = True
population_path = 'assets/HS-STAT-Population-of-Vermont-towns-1930-2019.xls'

# set the path to the characteristics data
characteristics_path = 'assets/Characteristic.csv'

# set the save path to the resulting cleaned chem data
chem_data_save_path = 'assets/chem_data_merged.csv'

# set the save path to the resulting survey data file
survay_save_path = 'assets/combined_tables.csv'

# set geojson path
geojson_path = 'assets/Lakes_Inventory.geojson'

In [99]:
URL_HEALTH = 'https://www.healthvermont.gov/sites/default/files/documents/xls/HS-STAT-Population-of-Vermont-towns-1930-2019.xls'

URL_LAKES = 'https://anrmaps.vermont.gov/arcgis/rest/services/Open_Data/OPENDATA_ANR_WATER_SP_NOCACHE_v2/MapServer/208/query?where=1%3D1&outFields=*&outSR=4326&f=json'

# This fuction downloads the geojson data from VERMONT OPEN GEODATA PORTAL.
def get_lakes_inventory_geojson(filename=geojson_path, url=URL_LAKES):
    if not os.path.exists(filename):
        urlretrieve(url, filename)
    data = gpd.read_file(r'assets/Lakes_Inventory.geojson')
    return data

# This fuction downloads the population data.
def get_pop_data(filename=population_path, url=URL_HEALTH):
    if not os.path.exists(filename):
        urlretrieve(url, filename)
    
    
    population = pd.read_excel(population_path, skiprows=4, index_col='CTC')
    # lets take the relevent columns from the population dataset and melt them so we ca easily join the dataframes
    population = population[['NAME'] + list(population.columns[-32:])]
    population.rename(columns={'NAME': 'Town'}, inplace=True)
    return population

In [100]:
# Download geojson data for ploting in project dashboard notebook

geojson = get_lakes_inventory_geojson()

In [101]:
# Some measurments have a very low frequency or have been measured only once or twice.
# Here we set a TRESHOLD parameter to cut off parameters with less than TRESHOLD measurments available in dataset.
 
# set treshold for number of measurments
TRESHOLD = 100

def get_frequent_measurments(df, TRESHOLD):
    counts = df.groupby(['CharacteristicID'])['Result'].count()
    mask = counts[counts.values <= TRESHOLD].index
    df = df[~df['CharacteristicID'].isin(mask)]
    return df

In [102]:
# combine all tables from Table Folder in assets folder

def merge_tables_folder(tables_folder):
    # get the first file as data frame to append to
    file_1 = os.listdir(tables_folder)[0]
    tables_df = pd.read_excel(os.path.join(tables_folder, file_1))
    
    # add file name to df
    tables_df['from_file'] = file_1
    
    # run through the rest of the files and append them to the data frame
    for file in os.listdir(tables_folder)[1:]:
        df = pd.read_excel(os.path.join(tables_folder, file))
        df['from_file'] = file
        tables_df = pd.concat((tables_df, df), axis=0)
    
    # drop OBJECTID column and reset index
    tables_df = tables_df.drop('OBJECTID', axis=1).reset_index(drop=True)
    return tables_df

In [103]:
combined_tables_df = merge_tables_folder(land_use_folder)

In [104]:
combined_tables_df.sample(5)

,Description,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Hay_acres,Ag_Crops_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file
725,VALLEY_Buffer100ftWBFL,13895.646044,2.521094e+05,50.627984,4.786980,0.057514,6.127648,0.227337,0.232217,0.231167,0.0,0.000000,0.000000,0.000000,0.000000,0.065753,0.266498,0.275028,0.482944,0.0,1.090223,0.121568,0.121568,25.748574,25.015865,50.764439,0.979810,4.294422,0.354032,5.628263,AOIs_VALLEY.xlsx
488,MIRROR_Flowline100ft,42781.368401,1.325336e+06,179.944950,59.001632,0.592311,83.976225,0.367013,2.632224,0.944003,0.0,1.894052,0.000000,0.262756,2.156808,0.698177,0.393719,1.042201,4.354130,0.0,6.488227,19.266377,19.266377,52.617790,127.942595,180.560385,24.672873,34.953664,22.028532,81.655069,AOIs_MIRROR.xlsx
210,FAIRLEE_Watershed,50603.138611,5.030529e+07,10829.864040,1421.133164,16.400050,53.445105,18.748968,43.052795,48.045799,0.0,776.234695,12.308547,1.683897,790.227139,17.177745,20.202026,53.100000,94.158554,0.0,184.638324,134.098025,134.098025,4962.198099,5875.960618,10838.158717,158.709943,603.461208,50.434183,812.605335,AOIs_FAIRLEE.xls
553,PEACHAM_Flowline100ft,18558.481251,5.575266e+05,106.633591,22.898232,0.592620,7.451142,0.000000,0.128433,0.038795,0.0,0.000000,0.000000,0.000000,0.000000,0.778316,0.000000,0.039687,1.272451,0.0,2.090454,10.390842,10.390842,58.325764,48.532512,106.858276,18.667997,46.251507,5.628293,70.547798,AOIs_PEACHAM.xlsx
482,MILLWINDSR_Watershed,64821.615624,1.133919e+08,22543.808487,4927.110796,28.375932,123.392870,74.172581,154.601799,168.242304,0.0,1772.160433,9.132234,1266.751360,3048.044027,30.929034,77.683037,179.634745,292.263092,0.0,580.509908,197.797632,197.797632,7147.891617,15418.066604,22565.958222,127.153824,915.775136,56.357625,1099.286585,AOIs_MILLWINDSR.xlsx


## Split the Description column to get LakeIDs
The Description column has the LakeID attached to a description. depending on the lake they may be LakeID then description or description then LakeID. the function below splits on an underscore and takes the part of the newly created list that is all uppercase as the LakeID and leaves the rest as the description.

In [105]:
# now I want to solit the description column in to two columns 
# one corresponds to the title area and the other corresponds to the description
# depending on the folder they are in different orders

def split_description(df):
    df['Description'] = df['Description'].str.split('_')
    df['LakeID'] = df['Description'].apply(lambda row: row[0] if row[0].isupper() else row[1])
    df['Description'] = df['Description'].apply(lambda row: row[0] if not row[0].isupper() else row[1])
    return df

In [106]:
combined_tables_df = split_description(combined_tables_df)
combined_tables_df.sample(5)

,Description,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Hay_acres,Ag_Crops_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file,LakeID
316,Watershed,11514.363073,2.817406e+06,639.571740,38.019822,0.000000,6.001563,1.129393,6.981581,4.491195,0.0,4.063963,0.0,0.292819,4.356783,0.000000,1.302965,5.486610,10.626632,0.0,17.416207,1.599808,1.599808,179.749615,460.199405,639.949019,11.490454,46.870442,4.263056,62.623952,AOIs_HIDDEN.xls,HIDDEN
414,Waterbody100ft,11539.717310,1.719462e+05,31.927881,8.629401,0.081977,1.432097,0.218688,0.031506,0.158147,0.0,0.967858,0.0,0.000000,0.967858,0.104026,0.244429,0.203791,0.082523,0.0,0.634770,2.847172,2.847172,21.905773,10.116286,32.022058,4.341580,12.596652,1.735981,18.674213,AOIs_LITTLEHOSMER.xls,LITTLEHOSMER
157,Waterbody100ft,10856.952346,1.633154e+05,36.182103,1.458537,0.036633,1.995188,0.353916,0.073205,0.256619,0.0,0.000000,0.0,0.000000,0.000000,0.051708,0.444215,0.427563,0.288215,0.0,1.211701,0.000000,0.000000,23.270779,13.029889,36.300668,0.045340,5.039647,0.332806,5.417792,AOIs_EASTLONG.xls,EASTLONG
327,Flowline100ft,1500.131136,4.176164e+04,4.545867,3.129955,0.000000,2.491621,0.000000,0.150858,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.151524,0.0,0.151524,0.279845,0.279845,2.970089,1.629099,4.599187,3.341299,2.996817,0.908847,7.246963,AOIs_HORSE.xls,HORSE
624,Waterbody100ft,33874.144341,5.114109e+05,67.381209,36.176913,0.862767,5.723075,7.730618,2.925785,5.554797,0.0,0.434995,0.0,0.000000,0.434995,1.015401,8.566333,6.137785,3.208734,0.0,18.928252,0.000000,0.000000,42.977654,24.727992,67.705646,1.900173,10.570155,0.023944,12.494273,AOIs_SEYMOUR.xls,SEYMOUR


## Merge relevant columns from the chem data and population datasets to survey data
For now I will just take the 'LakeID', 'Lat', 'Long', 'Town' from the chem data dataset. for other use later i will also extract the measurements when I know what year the land use survey was done. I will need to fix the LakeIDs in the combined_tables_df dataframe so that they match up with the LakeIDs in the chem dataset. This is the column that i will join on.

In [107]:
chem_data_df = pd.read_csv(chem_data_file_path)

In [108]:
# set TRSHOLD and run this if you want to exclude infrequent measurments
TRESHOLD = 100
chem_data_df = get_frequent_measurments(chem_data_df, TRESHOLD)

In [109]:
chem_data_df.sample(5)

,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode
46763,CHIPMAN,1,Pelagic,43.40890,-73.03170,Tinmouth,Laymon,1997-08-30 00:00:00,1,1710.0,BottleGrab,0.5,R1,Chla,NaN,2.81,Y,NaN,NaN,NaN
41534,CASPIAN,1,Pelagic,44.58897,-72.30878,Greensboro,SpringTP,1987-04-29 00:00:00,1,NaN,Hose,23.0,Reg,TP,NaN,6.00,Y,NaN,NaN,NaN
8844,BEEBE (HUBDTN),1,Pelagic,43.73440,-73.18330,Hubbardton,NLA,2007-08-02 00:00:00,1,1111.0,Hydrolab,3.0,Reg,TurbNTU,NaN,0.00,Y,NaN,NaN,NaN
23446,BRANCH,1,Pelagic,43.08079,-73.01964,Sunderland,AcidLake,1995-07-18 00:00:00,1,1300.0,PlasticKemm,1.0,Reg,DCl,NaN,0.30,Y,NaN,NaN,E
192696,PERCH (BENSON),1,Pelagic,43.75030,-73.28080,Benson,SpringTP,2011-04-24 00:00:00,1,1326.0,Hydrolab,6.0,Reg,pH,NaN,6.96,Y,NaN,NaN,NaN


In [110]:
if USE_POPULATION:    
    population = get_pop_data()

In [111]:
# now lets convert town names in the chem dataset to uppercase to match the town names in the population data set
chem_data_df['Town'] = chem_data_df['Town'].str.upper()
# we also need to extract the year from the chem data set so that we can join the population data
# to the chem data by year and Town
chem_data_df['VisitDate'] = pd.to_datetime(chem_data_df['VisitDate'])
chem_data_df['year'] = chem_data_df['VisitDate'].dt.year

In [112]:
chem_to_add = chem_data_df[['LakeID', 'Lat', 'Long', 'Town']].copy()
chem_to_add.drop_duplicates(inplace=True)

# get all lake ids that are also in usage survey and fix them
for lake_id in np.unique(chem_to_add['LakeID']):
    lake_id_fixed = re.sub('[()\s;]', '', lake_id)
    index = combined_tables_df[combined_tables_df['LakeID'] == lake_id_fixed].index
    combined_tables_df.loc[index, 'LakeID'] = lake_id


In [113]:
# merge with the chem_to_add data frame
new_data_df = combined_tables_df.merge(chem_to_add, how='left', on='LakeID')

# reorder columns
cols = list(new_data_df.columns)

to_front = ['LakeID', 'Description', 'Lat', 'Long', 'Town']
for col in to_front:
    cols.remove(col)

cols = to_front + cols
new_data_df = new_data_df.reindex(columns=cols)

new_data_df.head(5)

,LakeID,Description,Lat,Long,Town,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Hay_acres,Ag_Crops_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file
0,ABENAKI,Watershed,43.8303,-72.2361,THETFORD,9683.033459,2.335672e+06,532.331074,39.856060,0.254456,1.075957,0.821377,1.521425,1.297239,0.0,16.735400,0.0,0.0,16.735400,0.309766,0.894497,1.551584,3.145226,0.0,5.901073,0.0,0.0,266.678081,265.987483,532.665564,5.913928,29.521352,0.191924,35.627204,AOIs_ABENAKI.xls
1,ABENAKI,Flowline100ft,43.8303,-72.2361,THETFORD,6359.510779,1.864531e+05,41.212481,4.014035,0.000000,0.485129,0.028850,0.233947,0.091244,0.0,0.002157,0.0,0.0,0.002157,0.000000,0.030748,0.111193,0.337027,0.0,0.478968,0.0,0.0,19.499594,21.754112,41.253706,3.738571,3.145864,0.000000,6.884435,AOIs_ABENAKI.xls
2,ABENAKI,Waterbody100ft,43.8303,-72.2361,THETFORD,5631.483675,8.449326e+04,17.348810,2.912812,0.017421,0.545608,0.021066,0.000000,0.031506,0.0,0.000000,0.0,0.0,0.000000,0.067456,0.022888,0.044621,0.000000,0.0,0.134965,0.0,0.0,9.122220,8.232879,17.355099,2.099783,2.956447,0.182035,5.238265,AOIs_ABENAKI.xls
3,ABENAKI,Buffer100ftWBFL,43.8303,-72.2361,THETFORD,11439.865203,2.655565e+05,58.155297,6.164034,0.017421,0.867339,0.049915,0.233947,0.122749,0.0,0.002157,0.0,0.0,0.002157,0.067456,0.053636,0.155814,0.337027,0.0,0.613933,0.0,0.0,28.361753,29.837036,58.198789,4.854378,5.758748,0.182035,10.795161,AOIs_ABENAKI.xls
4,ABENAKI,Buffer250ftWaterbody,43.8303,-72.2361,THETFORD,5838.954038,2.125897e+05,44.545557,5.968327,0.017421,0.683245,0.293499,0.782087,0.236850,0.0,0.000000,0.0,0.0,0.000000,0.067456,0.316489,0.349577,1.192506,0.0,1.926027,0.0,0.0,21.194604,23.420545,44.615149,3.266376,7.531832,0.191924,10.990132,AOIs_ABENAKI.xls


In [114]:
# now since the land use survey was conducted from 2013 - 2016 I will merge the population data for 2016 joining on the town names
if USE_POPULATION:    
    pop2016 = population[['Town', 2016]].copy()
    pop2016.rename(columns={2016: '2016_population'}, inplace=True)
    new_data_df = new_data_df.merge(pop2016, how='left', on='Town')
    print('Final survey data DataFrame shape: ', new_data_df.shape)
    display(new_data_df.sample(5))

Final survey data DataFrame shape:  (773, 36)


,LakeID,Description,Lat,Long,Town,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Hay_acres,Ag_Crops_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file,2016_population
634,SHADOW (GLOVER),Waterbody100ft,44.66991,-72.22818,GLOVER,8272.732374,1.237534e+05,16.571356,7.104145,0.137205,1.089424,1.610939,1.816840,2.247111,0.000000,0.000000,0.000000,0.00000,0.000000,0.164473,1.880830,2.649203,2.314708,0.000000,7.009214,0.000000,0.000000,11.793413,4.869642,16.663055,0.000000,2.397023,0.000000,2.397023,AOIs_SHADOWGLOVER.xls,1086.0
617,SALEM,Watershed,44.93000,-72.10440,DERBY,116195.603967,3.412320e+08,61730.087327,17051.054207,215.648719,4101.659632,152.427213,439.435223,601.584350,28.241630,8859.842082,643.422942,0.47716,9503.742184,241.985406,157.988451,661.733747,568.773027,28.627862,1659.108494,1950.824986,1950.824986,20587.694778,41221.116243,61808.811021,2037.821333,13416.152568,934.857317,16388.831218,AOIs_SALEM.xlsx,4322.0
443,LYFORD,Flowline100ft,44.44499,-72.25037,WALDEN,188.199698,1.903840e+03,0.046456,0.333839,0.000000,0.090749,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003023,0.044944,0.047968,0.274825,0.000000,0.195203,0.470028,AOIs_LYFORD.xls,938.0
348,ISLAND,Waterbody100ft,44.80750,-71.87330,BRIGHTON,17015.451552,2.574722e+05,34.133849,13.774498,0.837933,3.437231,1.675804,1.277780,6.744051,1.735357,0.000000,0.000000,0.00000,0.000000,1.229194,1.833935,8.036131,1.675131,1.779039,14.553429,0.000000,0.000000,14.996454,19.239969,34.236423,1.677045,1.939548,0.035185,3.651778,AOIs_ISLAND.xls,1177.0
760,WOODWARD,Waterbody100ft,43.56500,-72.75970,PLYMOUTH,11265.775925,1.722640e+05,29.673295,8.879780,0.102363,0.556295,0.160804,3.008874,0.182549,0.000000,0.261202,0.000000,0.00000,0.261202,0.121394,0.179635,0.192327,3.367415,0.000000,3.860771,0.000000,0.000000,9.814316,19.918627,29.732943,0.042003,0.815254,0.000000,0.857257,AOIs_WOODWARD.xlsx,610.0


## Save the survey dataframe
Lets save the resulting dataframe as a .csv for later use

In [115]:
# save the data frame as a .csv
new_data_df.to_csv(survay_save_path)

## Merge the population dataset, the chem data dataset and the characteristics dataset
Here we will take the population dataset and melt it so that the years are also in the rows sp that we can merge on both town and year data.

In [116]:
# melt the population dataframe
if USE_POPULATION:
    population_1 = population.melt(id_vars='Town', var_name='year', value_name='population')
    chem_data_df = chem_data_df.merge(population_1, how='left', on=['Town', 'year'])
    display(chem_data_df.sample(5))

,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,year,population
80125,FAIRLEE,1,Pelagic,43.88294,-72.22941,THETFORD,Laymon,2000-06-07,1,1345.0,Hose,10.0,Reg,TP,NaN,8.00,Y,NaN,NaN,NaN,2000,2617.0
63346,DUNMORE,1,Pelagic,43.91220,-73.07640,SALISBURY,SpringTP,2004-04-20,1,1241.0,Hydrolab,20.0,Reg,TempC,NaN,5.30,Y,NaN,NaN,NaN,2004,1121.0
109027,HARRIMAN (NEWBRY),1,Pelagic,44.10304,-72.07947,NEWBURY,SpringTP,2010-04-02,1,1352.0,Kemmerer,3.0,Reg,TP,NaN,26.40,Y,NaN,NaN,NaN,2010,2216.0
195214,RAPONDA,1,Pelagic,42.87608,-72.81786,WILMINGTON,LayMon,2007-06-10,1,1530.0,BottleGrab,0.5,Reg,Chla,NaN,3.29,Y,NaN,NaN,NaN,2007,1976.0
278199,WILLOUGHBY,1,Pelagic,44.74886,-72.06116,WESTMORE,LayMon,2007-08-12,1,1615.0,Hose,10.0,R1,Chla,NaN,1.74,Y,NaN,NaN,NaN,2007,343.0


In [117]:
# Now so that we have measurements
characteristics = pd.read_csv(characteristics_path)
characteristics.columns = ['CharacteristicID', 'CharacteristicName', 'UnitCode', 'SampleFraction']
characteristics.head()

,CharacteristicID,CharacteristicName,UnitCode,SampleFraction
0,AshFreeDryMass,Ash Free Dry Mass,mg,NaN
1,BOD5,"Biological Oxygen Demand, 5 day",mg/l,NaN
2,BottomDepth,Bottom depth,m,NaN
3,BottomSecchi,Bottom secchi,NaN,NaN
4,CBOD5,"Carbonaceous Biological Oxygen Demand, 5 day",mg/l,NaN


In [118]:
# we are going to merge on CharacteristicID so to make sure that they are the same across dataframes
# we will make all if them uppercase in both dataframes
characteristics['CharacteristicID'] = characteristics['CharacteristicID'].str.upper()
chem_data_df['CharacteristicID'] = chem_data_df['CharacteristicID'].str.upper()

In [119]:
# Merge characteristics df
chem_data_df_final = chem_data_df.merge(characteristics, how='left', on='CharacteristicID')
print('Final chem data Data frame shape: ', chem_data_df_final.shape)
chem_data_df_final.sample(5)

Final chem data Data frame shape:  (284535, 25)


,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,year,population,CharacteristicName,UnitCode,SampleFraction
153393,LONG (WESTMR),1,Pelagic,44.75274,-72.01827,WESTMORE,SpringTP,2010-04-19,1,1119.0,Hydrolab,17.00,Reg,PH,NaN,7.53,Y,NaN,NaN,NaN,2010,350.0,pH,None,NaN
8886,BEEBE (HUBDTN),1,Pelagic,43.73440,-73.18330,HUBBARDTON,NLA,2007-08-02,1,1116.0,Hydrolab,11.00,Reg,COND,NaN,218.50,Y,NaN,NaN,NaN,2007,721.0,Conductivity,umho/cm,NaN
72780,ELFIN,1,Pelagic,43.46940,-72.98810,WALLINGFORD,Laymon,2016-06-14,1,745.0,Hose,6.00,R1,CHLA,NaN,10.12,Y,NaN,NaN,NaN,2016,2001.0,Chlorophyll-a,ug/l,NaN
101085,GROUT,1,Pelagic,43.04451,-72.94366,STRATTON,AcidLake,2014-04-29,1,1315.0,PlasticKemm,1.00,Reg,VISUALTOTCOLOR,NaN,25.00,Y,NaN,NaN,NaN,2014,212.0,Total Color measured using the visual method,PCU,Total
273598,WALLINGFORD,1,Pelagic,43.41137,-72.90856,WALLINGFORD,SpringTP,2015-05-26,1,1329.0,Hydrolab,0.25,Reg,DO,NaN,8.75,Y,NaN,NaN,NaN,2015,2026.0,Dissolved Oxygen,mg/l,NaN


In [120]:
# add in normalized results columns
for char_id in chem_data_df_final['CharacteristicID'].unique():
    df1 = chem_data_df_final[chem_data_df_final['CharacteristicID'] == char_id]
    chem_data_df_final.loc[df1.index, 'NormResult'] = MinMaxScaler().fit_transform(df1['Result'].values.reshape(-1, 1))
    
print(chem_data_df_final.shape)
chem_data_df_final.head()

(284535, 26)


,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,year,population,CharacteristicName,UnitCode,SampleFraction,NormResult
0,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1988-04-22,1,NaN,Secchi,NaN,Reg,SECCHI,NaN,2.8,Y,NaN,B,NaN,1988,2377.0,Secchi transparency,m,NaN,0.176694
1,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1988-04-22,1,NaN,Kemmerer,1.4,Reg,TP,NaN,9.0,Y,NaN,NaN,NaN,1988,2377.0,Total Phosphorus,ug/l,Total,0.004051
2,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1989-05-01,1,NaN,Secchi,NaN,Reg,SECCHI,NaN,2.3,Y,NaN,B,NaN,1989,2417.0,Secchi transparency,m,NaN,0.145028
3,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1989-05-01,1,NaN,Kemmerer,1.0,Reg,TP,NaN,11.0,Y,NaN,NaN,NaN,1989,2417.0,Total Phosphorus,ug/l,Total,0.005208
4,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1990-04-20,1,NaN,Secchi,NaN,Reg,SECCHI,NaN,3.1,Y,NaN,B,NaN,1990,2438.0,Secchi transparency,m,NaN,0.195693


## Save the chem data DataFrame
Lets save the resulting dataframe as a .csv for later use

In [121]:
chem_data_df_final.to_csv(chem_data_save_path)

## Documantation

In [122]:
%load_ext watermark
%watermark --iversions

sys      : 3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:36:06) [MSC v.1929 64 bit (AMD64)]
numpy    : 1.21.5
re       : 2.2.1
geopandas: 0.10.2
pandas   : 1.3.4

